In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-01-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-01-29 A        35.285362   37.650002   37.689999   36.310001   36.439999   
           AAL      37.260624   38.990002   39.090000   36.599998   37.520000   
           AAPL     22.126181   24.334999   24.334999   23.587500   23.697500   
           ABBV     38.896370   54.900002   55.310001   53.419998   53.419998   
           ABT      32.631168   37.849998   37.869999   36.680000   36.689999   
...                       ...         ...         ...         ...         ...   
2024-01-26 YUM     129.089996  129.089996  130.690002  128.669998  129.919998   
           ZBH     121.690002  121.690002  123.110001  121.570000  122.839996   
           ZBRA    252.169998  252.169998  258.420013  251.619995  256.980011   
           ZION     44.020000   44.020000   44.860001   43.959999   44.500000   
           ZTS     191.520004  191.520004  192.880005  188.970001  189.210007   

Price                   volume  
date       ticker               
2016-01-29 A         2959900.0  
           AAL      20957500.0  
           AAPL    257666000.0  
           ABBV     17768700.0  
           ABT      16124300.0  
...                        ...  
2024-01-26 YUM       1157000.0  
           ZBH        982800.0  
           ZBRA       268300.0  
           ZION      1504900.0  
           ZTS       1840200.0  

[994868 rows x 6 columns]

Calculate features and technical indicators for each stock.
    Garman-Klass Volatility
    RSI
    Bollinger Bands
    ATR
    MACD
    Dollar Volume

In [2]:
print(df.stack)

<bound method DataFrame.stack of Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-01-29 A        35.285362   37.650002   37.689999   36.310001   36.439999   
           AAL      37.260624   38.990002   39.090000   36.599998   37.520000   
           AAPL     22.126181   24.334999   24.334999   23.587500   23.697500   
           ABBV     38.896370   54.900002   55.310001   53.419998   53.419998   
           ABT      32.631168   37.849998   37.869999   36.680000   36.689999   
...                       ...         ...         ...         ...         ...   
2024-01-26 YUM     129.089996  129.089996  130.690002  128.669998  129.919998   
           ZBH     121.690002  121.690002  123.110001  121.570000  122.839996   
           ZBRA    252.169998  252.169998  258.420013  251.619995  256.980011   
           ZION     44.020000   44.020000   44.860001   43.959999   44.50000

In [3]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-01-29 A        35.285362   37.650002   37.689999   36.310001   36.439999   
           AAL      37.260624   38.990002   39.090000   36.599998   37.520000   
           AAPL     22.126181   24.334999   24.334999   23.587500   23.697500   
           ABBV     38.896370   54.900002   55.310001   53.419998   53.419998   
           ABT      32.631168   37.849998   37.869999   36.680000   36.689999   
...                       ...         ...         ...         ...         ...   
2024-01-26 YUM     129.089996  129.089996  130.690002  128.669998  129.919998   
           ZBH     121.690002  121.690002  123.110001  121.570000  122.839996   
           ZBRA    252.169998  252.169998  258.420013  251.619995  256.980011   
           ZION     44.020000   44.020000   44.860001   43.959999   44.500000   
           ZTS     191.520004  191.520004  192.880005  188.970001  189.210007   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2016-01-29 A         2959900.0          0.000295        NaN       NaN   
           AAL      20957500.0          0.002147        NaN       NaN   
           AAPL    257666000.0         -0.001332        NaN       NaN   
           ABBV     17768700.0         -0.038284        NaN       NaN   
           ABT      16124300.0         -0.004800        NaN       NaN   
...                        ...               ...        ...       ...   
2024-01-26 YUM       1157000.0          0.000105  50.708962  4.856013   
           ZBH        982800.0          0.000045  56.739863  4.794179   
           ZBRA       268300.0          0.000218  51.191561  5.479828   
           ZION      1504900.0          0.000160  57.948030  3.728311   
           ZTS       1840200.0          0.000153  52.795286  5.230000   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2016-01-29 A            NaN       NaN       NaN       NaN     104.441144  
           AAL          NaN       NaN       NaN       NaN     780.889526  
           AAPL         NaN       NaN       NaN       NaN    5701.164463  
           ABBV         NaN       NaN       NaN       NaN     691.137928  
           ABT          NaN       NaN       NaN       NaN     526.154748  
...                     ...       ...       ...       ...            ...  
2024-01-26 YUM     4.872770  4.889527 -0.004083  0.166119     149.357126  
           ZBH     4.811249  4.828319 -0.667646  0.369591     119.596934  
           ZBRA    5.545973  5.612118  0.054539 -0.021347      67.657211  
           ZION    3.786146  3.843982  0.453853  0.539881      66.245699  
           ZTS     5.267848  5.305696  0.707589 -0.325549     352.435112  

[994868 rows x 14 columns]

# Aggregate to monthly level and filter top 150 most liquid stocks for each month.
To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [4]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'Volume', 'Open',
                                                          'High', 'Low', 'Close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()
data

dollar_volume   adj close       close        high  \
date       ticker                                                      
2016-03-31 A           62.013607   37.347183   39.849998   40.099998   
           AAL        298.909259   39.297958   41.009998   41.680000   
           AAPL      3231.204706   24.908754   27.247499   27.475000   
           ABBV       256.185597   40.469223   57.119999   57.459999   
           ABT        229.241234   36.062397   41.830002   42.000000   
...                          ...         ...         ...         ...   
2024-01-31 YUM        196.809561  129.089996  129.089996  130.690002   
           ZBH        204.029004  121.690002  121.690002  123.110001   
           ZBRA        99.053216  252.169998  252.169998  258.420013   
           ZION       109.331953   44.020000   44.020000   44.860001   
           ZTS        363.109587  191.520004  191.520004  192.880005   

                          low        open       volume  garman_klass_vol  \
date       ticker                                                          
2016-03-31 A        39.660000   40.009998    3143500.0         -0.001771   
           AAL      40.980000   41.500000    8138000.0         -0.001005   
           AAPL     27.219999   27.430000  103553600.0         -0.003548   
           ABBV     56.820000   57.020000    4930300.0         -0.045347   
           ABT      41.509998   41.650002    5168400.0         -0.007947   
...                       ...         ...          ...               ...   
2024-01-31 YUM     128.669998  129.919998    1157000.0          0.000105   
           ZBH     121.570000  122.839996     982800.0          0.000045   
           ZBRA    251.619995  256.980011     268300.0          0.000218   
           ZION     43.959999   44.500000    1504900.0          0.000160   
           ZTS     188.970001  189.210007    1840200.0          0.000153   

                         rsi    bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                               
2016-03-31 A       57.954055  3.598942  3.632082  3.665222 -1.143102  0.165574  
           AAL     50.405323  3.680381  3.717086  3.753792  0.137565  0.644889  
           AAPL    71.146276  3.167556  3.215080  3.262603 -1.106125  0.028359  
           ABBV    55.020991  3.690339  3.712900  3.735461 -0.965906 -0.031223  
           ABT     68.904622  3.532534  3.575936  3.619339 -1.140258  0.346932  
...                      ...       ...       ...       ...       ...       ...  
2024-01-31 YUM     50.708962  4.856013  4.872770  4.889527 -0.004083  0.166119  
           ZBH     56.739863  4.794179  4.811249  4.828319 -0.667646  0.369591  
           ZBRA    51.191561  5.479828  5.545973  5.612118  0.054539 -0.021347  
           ZION    57.948030  3.728311  3.786146  3.843982  0.453853  0.539881  
           ZTS     52.795286  5.230000  5.267848  5.305696  0.707589 -0.325549  

[46973 rows x 14 columns]

In [5]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close       close        high         low        open  \
date       ticker                                                               
2017-02-28 AAL      44.898853   46.360001   46.720001   46.029999   46.320000   
           AAPL     31.969782   34.247501   34.360001   34.174999   34.270000   
           ABBV     45.488472   61.840000   62.400002   61.689999   62.110001   
           ABT      39.848415   45.080002   45.430000   44.830002   45.410000   
           ACN     109.569916  122.500000  123.089996  121.930000  122.750000   
...                       ...         ...         ...         ...         ...   
2024-01-31 VRTX    430.170013  430.170013  432.190002  428.140015  431.029999   
           VZ       42.400002   42.400002   42.490002   42.099998   42.290001   
           WFC      49.969025   50.320000   50.500000   49.770000   49.840000   
           WMT     164.270004  164.270004  164.380005  162.660004  163.050003   
           XOM     102.051567  103.000000  103.080002  101.190002  101.970001   

                       volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2017-02-28 AAL      4935600.0         -0.000264  50.168970  3.767394   
           AAPL    93931600.0         -0.001850  84.344866  3.403955   
           ABBV     6756100.0         -0.037405  53.680496  3.810368   
           ABT     14211400.0         -0.006505  71.877275  3.624358   
           ACN      1744900.0         -0.004939  63.223089  4.614227   
...                       ...               ...        ...       ...   
2024-01-31 VRTX      944500.0          0.000043  64.519320  6.005536   
           VZ      20367200.0          0.000040  70.186636  3.621979   
           WFC     19211100.0          0.000103  63.682816  3.855901   
           WMT      5248800.0          0.000034  60.845739  5.058206   
           XOM     20817200.0          0.000171  55.632470  4.560950   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2017-02-28 AAL     3.816059  3.864723  0.232846  0.085738  
           AAPL    3.465752  3.527550 -1.183687  0.184435  
           ABBV    3.829030  3.847692 -1.520582 -0.155108  
           ABT     3.679207  3.734055 -1.270983  0.663307  
           ACN     4.672869  4.731512 -1.104399  0.177282  
...                     ...       ...       ...       ...  
2024-01-31 VRTX    6.054672  6.103808  0.784810  2.545905  
           VZ      3.697084  3.772190  0.079807  2.476290  
           WFC     3.900935  3.945969 -0.539880  0.597304  
           WMT     5.084690  5.111174 -0.074353  0.916661  
           XOM     4.603986  4.647022  0.181979 -0.442341  

[12516 rows x 13 columns]

# Calculate Monthly Returns for different time horizons as features.
To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.

In [13]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
data

adj close       close        high         low        open  \
date       ticker                                                               
2018-02-28 AAL      52.977318   54.250000   54.599998   53.639999   54.410000   
           AAPL     42.226002   44.529999   45.154999   44.512501   44.814999   
           ABBV     88.056595  115.830002  120.000000  115.779999  119.000000   
           ABT      54.474922   60.330002   61.119999   60.180000   60.669998   
           ACN     146.918533  161.009995  164.389999  160.949997  162.899994   
...                       ...         ...         ...         ...         ...   
2024-01-31 VRTX    430.170013  430.170013  432.190002  428.140015  431.029999   
           VZ       42.400002   42.400002   42.490002   42.099998   42.290001   
           WFC      49.969025   50.320000   50.500000   49.770000   49.840000   
           WMT     164.270004  164.270004  164.380005  162.660004  163.050003   
           XOM     102.051567  103.000000  103.080002  101.190002  101.970001   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2018-02-28 AAL       4343800.0         -0.000118  55.685241  3.875603   
           AAPL    151128400.0         -0.001265  59.836091  3.615828   
           ABBV      6660000.0         -0.034391  56.507209  4.420361   
           ABT       8325700.0         -0.004361  55.823920  3.947487   
           ACN       2002200.0         -0.003895  54.875606  4.932437   
...                        ...               ...        ...       ...   
2024-01-31 VRTX       944500.0          0.000043  64.519320  6.005536   
           VZ       20367200.0          0.000040  70.186636  3.621979   
           WFC      19211100.0          0.000103  63.682816  3.855901   
           WMT       5248800.0          0.000034  60.845739  5.058206   
           XOM      20817200.0          0.000171  55.632470  4.560950   

                     bb_mid   bb_high       atr      macd  return_1m  \
date       ticker                                                      
2018-02-28 AAL     3.945411  4.015218  1.360197  0.425211   0.000632   
           AAPL    3.704858  3.793888 -0.787408 -0.023605   0.068185   
           ABBV    4.480013  4.539666  1.939831  1.168699   0.032169   
           ABT     3.999472  4.051457 -0.601833 -0.045659  -0.029440   
           ACN     4.981648  5.030859 -0.512395  0.111843   0.001929   
...                     ...       ...       ...       ...        ...   
2024-01-31 VRTX    6.054672  6.103808  0.784810  2.545905   0.057214   
           VZ      3.697084  3.772190  0.079807  2.476290   0.128932   
           WFC     3.900935  3.945969 -0.539880  0.597304   0.022349   
           WMT     5.084690  5.111174 -0.074353  0.916661   0.041992   
           XOM     4.603986  4.647022  0.181979 -0.442341   0.030206   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2018-02-28 AAL      0.022092   0.024888   0.033351   0.013426    0.013883  
           AAPL     0.028019   0.013390   0.015155   0.018521    0.023458  
           ABBV     0.098315   0.063744   0.076940   0.065862    0.050452  
           ABT      0.030620   0.024517   0.030254   0.033180    0.026397  
           ACN      0.025541   0.028460   0.036957   0.030116    0.024744  
...                      ...        ...        ...        ...         ...  
2024-01-31 VRTX     0.101088   0.059091   0.033823   0.026237    0.024138  
           VZ       0.060583   0.066071   0.043639   0.016077    0.007677  
           WFC      0.062311   0.084792   0.017307   0.029339    0.008523  
           WMT      0.029091   0.002990   0.005761   0.010665    0.012388  
           XOM      0.001265  -0.006045  -0.003838  -0.012469   -0.007089  

[10345 rows x 19 columns]

In [18]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-02-28 AAL    -0.0365  0.0032 -0.0104  0.0052 -0.0237   0.000632
           AAPL   -0.0365  0.0032 -0.0104  0.0052 -0.0237   0.068185
           ABBV   -0.0365  0.0032 -0.0104  0.0052 -0.0237   0.032169
           ABT    -0.0365  0.0032 -0.0104  0.0052 -0.0237  -0.029440
           ACN    -0.0365  0.0032 -0.0104  0.0052 -0.0237   0.001929
...                   ...     ...     ...     ...     ...        ...
2023-12-31 VRTX    0.0485  0.0732  0.0494 -0.0307  0.0132   0.146783
           VZ      0.0485  0.0732  0.0494 -0.0307  0.0132  -0.016436
           WFC     0.0485  0.0732  0.0494 -0.0307  0.0132   0.103835
           WMT     0.0485  0.0732  0.0494 -0.0307  0.0132   0.016350
           XOM     0.0485  0.0732  0.0494 -0.0307  0.0132  -0.026864

[10200 rows x 6 columns]

In [19]:
observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-02-28 AAL    -0.0365  0.0032 -0.0104  0.0052 -0.0237   0.000632
           AAPL   -0.0365  0.0032 -0.0104  0.0052 -0.0237   0.068185
           ABBV   -0.0365  0.0032 -0.0104  0.0052 -0.0237   0.032169
           ABT    -0.0365  0.0032 -0.0104  0.0052 -0.0237  -0.029440
           ACN    -0.0365  0.0032 -0.0104  0.0052 -0.0237   0.001929
...                   ...     ...     ...     ...     ...        ...
2023-12-31 VRTX    0.0485  0.0732  0.0494 -0.0307  0.0132   0.146783
           VZ      0.0485  0.0732  0.0494 -0.0307  0.0132  -0.016436
           WFC     0.0485  0.0732  0.0494 -0.0307  0.0132   0.103835
           WMT     0.0485  0.0732  0.0494 -0.0307  0.0132   0.016350
           XOM     0.0485  0.0732  0.0494 -0.0307  0.0132  -0.026864

[10160 rows x 6 columns]

In [20]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas 

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2018-02-28 AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2023-12-31 VRTX    0.360415  0.009107 -0.291635 -0.024355  0.716337
           VZ      0.441509 -0.657971  0.529770  0.182677 -0.222911
           WFC     1.028875  0.200398  1.979783 -0.295540 -1.477349
           WMT     0.515456  0.193586 -0.756227  0.638941  0.818299
           XOM     0.738269 -0.467574  1.071821  0.523877  0.308784

[10160 rows x 5 columns]

In [21]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10184 entries, (Timestamp('2018-02-28 00:00:00'), 'AAL') to (Timestamp('2024-01-31 00:00:00'), 'XOM')
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   close             10184 non-null  float64
 1   high              10184 non-null  float64
 2   low               10184 non-null  float64
 3   open              10184 non-null  float64
 4   volume            10184 non-null  float64
 5   garman_klass_vol  10184 non-null  float64
 6   rsi               10184 non-null  float64
 7   bb_low            10184 non-null  float64
 8   bb_mid            10184 non-null  float64
 9   bb_high           10184 non-null  float64
 10  atr               10184 non-null  float64
 11  macd              10184 non-null  float64
 12  return_1m         10184 non-null  float64
 13  return_2m         10184 non-null  float64
 14  return_3m         10184 non-null  float64
 15  return_6m  

## For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
# K-Means Clustering
You may want to initialize predefined centroids for each cluster based on your research.

For visualization purpose of this tutorial we will initially rely on the ‘k-means++’ initialization.

Then we will pre-define our centroids for each cluster.